In [1]:
import exploding_kittens as expl
import random

# GAME MECHANICS

##### 1. START OF TURN:
- PLAYER THINKING -> PLAYER ACTION
- DISCARD PILE UPDATED
- DRAW (END TURN)

In [2]:
class Game:
    """
    Class for the main functionality of the game. Upon initialisation,
    the deck is created and each player is dealt a hand of 5 cards
    plus one Defuse card. Then the remaining Defuse cards and the
    Exploding Kittens are shuffled into the deck.
    """

    def __init__(self, players=("Player 1", "Player 2")):
        self.deck = expl.Deck(num_of_players=len(players))
        self.discard_pile = []
        self.dead_players = []
        self.players = []
        for pl in players:
            self.players.append(expl.Player(pl))
            self.players[-1].deal_hand(
                [self.deck.cards.pop(0) for _ in range(5)] + [expl.Defuse()]
            )
            self.players[-1].hand.sort(key=lambda x: x.id)
        self.deck.add_defuse_and_exploding_to_deck()

    def update_discard_pile(self, played_card):
        if played_card is not None:
            self.discard_pile += [played_card]

    def execute_all_turns_for_a_player(self):
        """
        For the current player:

            while the player is still required to draw:
                while the player wants to play a card:
                    - Choose what card to play
                    - If a card is chosen
                        - Update the discard pile
                        - Perform the card's action
                    - Otherwise, set the stopping criteria to True
                - The player draws a card to end their turn
                - Check if the player exploded and resolve it
        """
        while self.cards_to_draw != 0:
            end_turn = False

            while not end_turn:
                played_card = self.current_player.choose_card_to_play()
                if played_card is not None:
                    self.update_discard_pile(played_card)
                    played_card.action(self)  # Pass the game class into the Cards class
                else:
                    end_turn = True

            # TODO: Need a check here for the case of playing a Skip or an Attack
            # e.g. When a player has 1 more card to draw and plays a Skip, 
            # should we set self.cards_to_draw=0?
            # Similarly if the same player plays an Attack, should we set
            # self.cards_to_draw = 2 OR self.next_player_cards_to_draw = 0
            
            drawn_card = self.current_player.draw_to_end_turn(self)
            if drawn_card.id == 0:
                exploded = self.current_player.explode()
                if exploded:
                    self.dead_players.append(self.current_player)
                    self.players.pop(self.current_player.id)
                    self.update_discard_pile(drawn_card)

            self.cards_to_draw -= 1


    def play_game(self):
        end_of_game = False
        self.turn_count = 0
        self.cards_to_draw = 1
        while not end_of_game:
            player_id = turn_count % len(self.players)
            self.current_player = self.players[player_id]
            self.current_player.id = player_id

            self.execute_all_turns_for_a_player()
            self.cards_to_draw = 1

            if len(self.players) == 1:
                end_of_game = True

            turn_count += 1


In [3]:
game = Game(players=["Mixas", "Vasos", "Ponjos"])

In [4]:
game.play_game()

In [5]:
game.players

[Mixas]

In [6]:
game.dead_players

[Vasos, Ponjos]

In [7]:
game.discard_pile

[Favor,
 Skip,
 See The Future,
 Shuffle,
 See The Future,
 Hairy Potato Cat,
 Attack,
 Exploding Kitten,
 Beard Cat,
 Skip,
 Favor,
 Tacocat,
 See The Future,
 Nope,
 Nope,
 Exploding Kitten]

In [8]:
game.dead_players[1].hand

[Defuse,
 Catermelon,
 Catermelon,
 Hairy Potato Cat,
 Beard Cat,
 Attack,
 Skip,
 Shuffle,
 Shuffle,
 Nope,
 Tacocat,
 Shuffle,
 Rainbow Cat,
 Nope,
 Exploding Kitten]

In [9]:
game.players[0].hand

[Defuse, Attack, Defuse, Catermelon, Favor, Rainbow Cat]